# Chapter 6: Split-Ring Resonators

**PyMetaLearn - Metamaterial Physics Workbook**

---

## Learning Objectives

By the end of this chapter, you will:
- Understand how to create artificial magnetic resonance
- Derive the effective permeability of split-ring resonators (SRRs)
- Calculate the magnetic resonance frequency
- Design SRRs to achieve μ < 0
- Combine SRRs with wire media for double-negative materials

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Arc, Rectangle, FancyArrowPatch
from matplotlib.collections import PatchCollection
from IPython.display import display, Markdown, Math

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Physical constants
c = 3e8            # Speed of light (m/s)
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space (H/m)
epsilon_0 = 8.854e-12    # Permittivity of free space (F/m)

print("Libraries loaded!")

---

## 6.1 The Challenge: Creating μ < 0

### Natural Magnetism

In natural materials:
- **Diamagnetic**: $\mu_r \approx 1 - 10^{-5}$ (slightly less than 1)
- **Paramagnetic**: $\mu_r \approx 1 + 10^{-5}$ (slightly more than 1)
- **Ferromagnetic**: $\mu_r >> 1$ (but only at low frequencies)

At optical and even microwave frequencies, **μ ≈ 1 for all natural materials**!

### Why?

Magnetic response comes from:
- Electron orbital motion
- Electron spin

These are too slow to respond at high frequencies.

### Pendry's Solution (1999)

**Sir John Pendry** proposed using metallic resonant structures that act as artificial "magnetic atoms" — the **Split-Ring Resonator (SRR)**.

In [ ]:
# Visualize the split-ring resonator geometry
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# --- Single SRR ---
ax1 = axes[0]

# Draw outer ring with gap
theta1 = np.linspace(15, 345, 100) * np.pi / 180
r_out = 1.0
ax1.plot(r_out * np.cos(theta1), r_out * np.sin(theta1), 'b-', linewidth=8)

# Draw inner ring with gap (opposite side)
theta2 = np.linspace(195, 525, 100) * np.pi / 180
r_in = 0.7
ax1.plot(r_in * np.cos(theta2), r_in * np.sin(theta2), 'b-', linewidth=8)

# Labels
ax1.annotate('Gap', xy=(0.95, 0.15), fontsize=11, color='red')
ax1.annotate('Gap', xy=(-0.75, -0.1), fontsize=11, color='red')

# Magnetic field direction
ax1.annotate('', xy=(0, 0.2), xytext=(0, -0.2),
            arrowprops=dict(arrowstyle='->', lw=2, color='green'))
ax1.text(0.15, 0, 'B', fontsize=14, color='green', fontweight='bold')

# Current direction
ax1.annotate('', xy=(0.4, 0.85), xytext=(0.7, 0.7),
            arrowprops=dict(arrowstyle='->', lw=2, color='orange', connectionstyle='arc3,rad=-0.3'))
ax1.text(0.8, 0.6, 'I', fontsize=14, color='orange', fontweight='bold')

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_aspect('equal')
ax1.set_title('Split-Ring Resonator (SRR)', fontsize=14)
ax1.grid(True, alpha=0.3)

# --- SRR Array ---
ax2 = axes[1]

for i in range(3):
    for j in range(3):
        x_center = i * 2.5 - 2.5
        y_center = j * 2.5 - 2.5
        
        # Outer ring
        theta1 = np.linspace(15, 345, 50) * np.pi / 180
        r_out = 0.8
        ax2.plot(x_center + r_out * np.cos(theta1), 
                y_center + r_out * np.sin(theta1), 'b-', linewidth=3)
        
        # Inner ring
        theta2 = np.linspace(195, 525, 50) * np.pi / 180
        r_in = 0.5
        ax2.plot(x_center + r_in * np.cos(theta2), 
                y_center + r_in * np.sin(theta2), 'b-', linewidth=3)

ax2.set_xlim(-4.5, 4.5)
ax2.set_ylim(-4.5, 4.5)
ax2.set_aspect('equal')
ax2.set_title('SRR Array (Metamaterial)', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The SRR is essentially a small LC circuit!")
print("• Rings provide inductance (L)")
print("• Gaps provide capacitance (C)")
print("• Resonance at ω₀ = 1/√(LC)")

---

## 6.2 The LC Circuit Model

An SRR behaves as an LC resonator:

### Inductance
The circular current path creates inductance:
$$L \approx \mu_0 \pi r^2 / h$$

Where:
- $r$ = ring radius
- $h$ = ring height (for 3D) or unit cell size

### Capacitance
The gaps act as capacitors:
$$C \approx \varepsilon_0 \varepsilon_r \frac{A_{gap}}{d_{gap}}$$

Where:
- $A_{gap}$ = gap area
- $d_{gap}$ = gap width

### Resonance Frequency

$$\boxed{\omega_0 = \frac{1}{\sqrt{LC}}}$$

By adjusting the geometry, we can tune $\omega_0$ from MHz to THz!

In [ ]:
def srr_resonance_frequency(r, d_gap, w, eps_gap=1.0):
    """
    Estimate SRR resonance frequency.
    
    r: ring radius (m)
    d_gap: gap width (m)
    w: ring width (m)
    eps_gap: gap dielectric constant
    
    This is a simplified model!
    """
    # Inductance (simplified circular loop)
    L = mu_0 * r * (np.log(8 * r / w) - 2)
    
    # Capacitance (parallel plate approximation)
    # For double SRR, there are two gaps in series
    h = w  # Assume square cross-section
    C_gap = epsilon_0 * eps_gap * w * h / d_gap
    C_total = C_gap / 2  # Two gaps in series
    
    # Resonance frequency
    omega_0 = 1 / np.sqrt(L * C_total)
    f_0 = omega_0 / (2 * np.pi)
    
    return f_0, L, C_total

# Example: Design SRR for different frequencies
print("SRR Design Examples")
print("=" * 60)

designs = [
    {'name': 'Microwave (10 GHz)', 'r': 2e-3, 'd_gap': 0.2e-3, 'w': 0.5e-3},
    {'name': 'mm-Wave (100 GHz)', 'r': 200e-6, 'd_gap': 20e-6, 'w': 50e-6},
    {'name': 'THz (1 THz)', 'r': 20e-6, 'd_gap': 2e-6, 'w': 5e-6},
    {'name': 'Infrared (30 THz)', 'r': 1e-6, 'd_gap': 100e-9, 'w': 200e-9},
]

print(f"{'Design':<25} {'Radius':<12} {'Gap':<12} {'f₀':<15}")
print("-" * 60)

for d in designs:
    f_0, L, C = srr_resonance_frequency(d['r'], d['d_gap'], d['w'])
    
    # Format units nicely
    if f_0 > 1e12:
        f_str = f"{f_0/1e12:.1f} THz"
    elif f_0 > 1e9:
        f_str = f"{f_0/1e9:.1f} GHz"
    else:
        f_str = f"{f_0/1e6:.1f} MHz"
    
    r_str = f"{d['r']*1e6:.0f} μm" if d['r'] > 1e-6 else f"{d['r']*1e9:.0f} nm"
    gap_str = f"{d['d_gap']*1e6:.0f} μm" if d['d_gap'] > 1e-6 else f"{d['d_gap']*1e9:.0f} nm"
    
    print(f"{d['name']:<25} {r_str:<12} {gap_str:<12} {f_str:<15}")

print("\n💡 Smaller SRRs resonate at higher frequencies!")

---

## 6.3 Effective Permeability of SRR Medium

When many SRRs are arranged in a periodic array, the collective response can be described by an **effective permeability**.

### Pendry's Result (1999)

$$\boxed{\mu_{eff}(\omega) = 1 - \frac{F \omega^2}{\omega^2 - \omega_0^2 + i\gamma\omega}}$$

Where:
- $F$ = filling fraction (geometrical factor, $0 < F < 1$)
- $\omega_0$ = magnetic resonance frequency
- $\gamma$ = damping (from resistive losses)

### Key Features

| Frequency Region | μ_eff |
|-----------------|-------|
| $\omega << \omega_0$ | $\mu_{eff} \approx 1$ |
| $\omega = \omega_0$ | Resonance (absorption) |
| $\omega_0 < \omega < \omega_0/\sqrt{1-F}$ | $\mu_{eff} < 0$ ! |
| $\omega >> \omega_0/\sqrt{1-F}$ | $\mu_{eff} \to 1$ |

In [ ]:
def srr_effective_permeability(omega, omega_0, F, gamma):
    """
    Calculate effective permeability of SRR medium.
    
    omega: angular frequency
    omega_0: resonance frequency
    F: filling fraction (0 < F < 1)
    gamma: damping rate
    """
    mu_eff = 1 - F * omega**2 / (omega**2 - omega_0**2 + 1j * gamma * omega)
    return mu_eff

# Parameters
omega_0 = 1.0  # Normalized
F = 0.3        # 30% filling fraction
gamma = 0.05   # Low damping

omega = np.linspace(0.01, 2.5, 1000)
mu = srr_effective_permeability(omega, omega_0, F, gamma)

# Calculate magnetic plasma frequency
omega_mp = omega_0 / np.sqrt(1 - F)

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Real part
ax1 = axes[0]
ax1.plot(omega, mu.real, 'b-', linewidth=2)
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax1.axhline(y=1, color='gray', linestyle='--', label='μ = 1')
ax1.axvline(x=omega_0, color='red', linestyle=':', linewidth=2, label=r'$\omega_0$')
ax1.axvline(x=omega_mp, color='green', linestyle=':', linewidth=2, label=r'$\omega_{mp}$')

# Shade μ < 0 region
ax1.fill_between(omega, mu.real, 0, where=(mu.real < 0), 
                 alpha=0.3, color='red', label='μ < 0 region')

ax1.set_xlabel(r'$\omega / \omega_0$', fontsize=14)
ax1.set_ylabel(r"$\mu'$ (Real Part)", fontsize=14)
ax1.set_title('Effective Permeability of SRR Medium', fontsize=14)
ax1.set_xlim(0, 2.5)
ax1.set_ylim(-5, 3)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Imaginary part (absorption)
ax2 = axes[1]
ax2.plot(omega, mu.imag, 'r-', linewidth=2)
ax2.axvline(x=omega_0, color='red', linestyle=':', linewidth=2, label=r'$\omega_0$')
ax2.set_xlabel(r'$\omega / \omega_0$', fontsize=14)
ax2.set_ylabel(r"$\mu''$ (Imaginary Part)", fontsize=14)
ax2.set_title('Magnetic Absorption', fontsize=14)
ax2.set_xlim(0, 2.5)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Magnetic resonance frequency: ω₀ = {omega_0:.2f}")
print(f"Magnetic plasma frequency: ω_mp = {omega_mp:.2f}")
print(f"μ < 0 region: {omega_0:.2f} < ω < {omega_mp:.2f}")

---

## 6.4 Effect of Design Parameters

The SRR response depends critically on:

1. **Filling fraction F**: Controls the bandwidth of μ < 0 region
2. **Damping γ**: Controls losses and affects minimum μ
3. **Resonance frequency ω₀**: Set by geometry (LC)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
omega = np.linspace(0.01, 2.5, 500)

# Effect of filling fraction
ax1 = axes[0, 0]
F_values = [0.1, 0.2, 0.3, 0.5]
for F in F_values:
    mu = srr_effective_permeability(omega, 1.0, F, 0.05)
    ax1.plot(omega, mu.real, linewidth=2, label=f'F = {F}')
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax1.set_xlabel(r'$\omega / \omega_0$', fontsize=12)
ax1.set_ylabel(r"$\mu'$", fontsize=12)
ax1.set_title('Effect of Filling Fraction F', fontsize=12)
ax1.set_ylim(-8, 3)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Effect of damping
ax2 = axes[0, 1]
gamma_values = [0.01, 0.05, 0.1, 0.2]
for gamma in gamma_values:
    mu = srr_effective_permeability(omega, 1.0, 0.3, gamma)
    ax2.plot(omega, mu.real, linewidth=2, label=f'γ = {gamma}')
ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax2.set_xlabel(r'$\omega / \omega_0$', fontsize=12)
ax2.set_ylabel(r"$\mu'$", fontsize=12)
ax2.set_title('Effect of Damping γ', fontsize=12)
ax2.set_ylim(-8, 3)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Absorption for different damping
ax3 = axes[1, 0]
for gamma in gamma_values:
    mu = srr_effective_permeability(omega, 1.0, 0.3, gamma)
    ax3.plot(omega, mu.imag, linewidth=2, label=f'γ = {gamma}')
ax3.set_xlabel(r'$\omega / \omega_0$', fontsize=12)
ax3.set_ylabel(r"$\mu''$", fontsize=12)
ax3.set_title('Absorption vs Damping', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)

# μ < 0 bandwidth vs F
ax4 = axes[1, 1]
F_range = np.linspace(0.05, 0.8, 100)
bandwidth = []
for F in F_range:
    omega_mp = 1.0 / np.sqrt(1 - F)
    bw = (omega_mp - 1.0) / 1.0 * 100  # Percentage
    bandwidth.append(bw)

ax4.plot(F_range * 100, bandwidth, 'b-', linewidth=2)
ax4.set_xlabel('Filling Fraction F (%)', fontsize=12)
ax4.set_ylabel('μ < 0 Bandwidth (%)', fontsize=12)
ax4.set_title('Negative-μ Bandwidth vs Filling Fraction', fontsize=12)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insights:")
print("• Higher F → wider μ < 0 bandwidth")
print("• Lower γ → sharper resonance, more negative minimum μ")
print("• Trade-off: wider bandwidth requires larger SRRs")

---

## 6.5 Physical Origin: Induced Currents

### How SRRs Respond to Magnetic Fields

1. **Incident magnetic field** $B(t) = B_0 e^{-i\omega t}$ threads through the SRR
2. By Faraday's law, an **EMF is induced** in the ring
3. Current flows in the conducting rings
4. The current creates its own **magnetic moment** $m$
5. This moment can **oppose** the applied field (diamagnetic response)!

### Near Resonance

- Current amplitude is **amplified** by the LC resonance
- The induced moment can be **larger** than what the applied field produces in vacuum
- Result: **μ < 0** !

In [ ]:
# Visualize the current response
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

omega_values = [0.5, 1.0, 1.3]  # Below, at, above resonance
omega_0 = 1.0

for ax, omega in zip(axes, omega_values):
    # Draw SRR
    theta = np.linspace(15, 345, 100) * np.pi / 180
    ax.plot(np.cos(theta), np.sin(theta), 'b-', linewidth=8)
    
    # Calculate response
    mu = srr_effective_permeability(omega, omega_0, 0.3, 0.05)
    
    # Show induced magnetic moment (simplified)
    m = (1 - mu.real)  # Proportional to induced moment
    
    # Applied field (into page)
    ax.text(0, 0, '⊗', fontsize=40, ha='center', va='center', color='green')
    ax.text(0, -0.3, 'B_applied', fontsize=10, ha='center', color='green')
    
    # Current direction arrow
    if m > 0:
        style = 'arc3,rad=-0.3'
        current_color = 'red'
        label = 'CCW'
    else:
        style = 'arc3,rad=0.3'
        current_color = 'orange'
        label = 'CW'
    
    arrow_scale = min(abs(m / 2), 1)
    ax.annotate('', xy=(0.8, 0.5), xytext=(0.5, 0.8),
                arrowprops=dict(arrowstyle='->', lw=2 + arrow_scale * 3, 
                               color=current_color, connectionstyle=style))
    
    # Frequency label
    if omega < omega_0:
        region = f'Below resonance\nμ = {mu.real:.2f}'
    elif omega == omega_0:
        region = f'At resonance\nμ = {mu.real:.2f}'
    else:
        region = f'Above resonance\nμ = {mu.real:.2f}'
    
    ax.set_title(f'ω/ω₀ = {omega}\n{region}', fontsize=12)
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Above resonance (ω > ω₀): Current and field are out of phase →  μ < 0!")

---

## 6.6 SRR Variations

Many SRR designs have been developed:

### Classic Double SRR (Pendry 1999)
- Two concentric rings with gaps on opposite sides
- Increased capacitance → lower frequency

### Single Ring SRR
- Simpler fabrication
- Higher resonance frequency

### Broadside-Coupled SRR (BC-SRR)
- Two rings on opposite sides of substrate
- Better isotropy

### Omega-Shaped Resonator
- Reduced bianisotropy
- More isotropic response

### Complementary SRR (CSRR)
- Babinet complement: slots instead of metal
- Electric resonance instead of magnetic

In [ ]:
# Show different SRR variations
fig, axes = plt.subplots(2, 3, figsize=(14, 9))

def draw_classic_srr(ax, title):
    theta1 = np.linspace(15, 345, 100) * np.pi / 180
    theta2 = np.linspace(195, 525, 100) * np.pi / 180
    ax.plot(np.cos(theta1), np.sin(theta1), 'b-', linewidth=6)
    ax.plot(0.6 * np.cos(theta2), 0.6 * np.sin(theta2), 'b-', linewidth=6)
    ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.axis('off')

def draw_single_srr(ax, title):
    theta = np.linspace(15, 345, 100) * np.pi / 180
    ax.plot(np.cos(theta), np.sin(theta), 'b-', linewidth=8)
    ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.axis('off')

def draw_omega(ax, title):
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'b-', linewidth=6)
    ax.plot([0, 0], [-1, -1.5], 'b-', linewidth=6)
    ax.plot([-0.3, 0.3], [-1.5, -1.5], 'b-', linewidth=6)
    ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.8, 1.3)
    ax.axis('off')

def draw_square_srr(ax, title):
    # Outer square with gap
    ax.plot([-1, 1], [1, 1], 'b-', linewidth=6)
    ax.plot([1, 1], [1, -1], 'b-', linewidth=6)
    ax.plot([1, 0.1], [-1, -1], 'b-', linewidth=6)
    ax.plot([-0.1, -1], [-1, -1], 'b-', linewidth=6)
    ax.plot([-1, -1], [-1, 1], 'b-', linewidth=6)
    
    # Inner square with gap
    ax.plot([-0.6, 0.6], [0.6, 0.6], 'b-', linewidth=6)
    ax.plot([0.6, 0.6], [0.6, -0.6], 'b-', linewidth=6)
    ax.plot([0.6, -0.6], [-0.6, -0.6], 'b-', linewidth=6)
    ax.plot([-0.6, -0.6], [-0.6, 0.1], 'b-', linewidth=6)
    ax.plot([-0.6, -0.6], [0.3, 0.6], 'b-', linewidth=6)
    ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.axis('off')

def draw_symmetic_srr(ax, title):
    # Two gaps, 180° apart
    theta1 = np.linspace(10, 170, 50) * np.pi / 180
    theta2 = np.linspace(190, 350, 50) * np.pi / 180
    ax.plot(np.cos(theta1), np.sin(theta1), 'b-', linewidth=8)
    ax.plot(np.cos(theta2), np.sin(theta2), 'b-', linewidth=8)
    ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.axis('off')

def draw_csrr(ax, title):
    # CSRR: slots in metal plane
    from matplotlib.patches import Wedge
    ax.add_patch(plt.Rectangle((-1.2, -1.2), 2.4, 2.4, color='blue', alpha=0.3))
    theta = np.linspace(0, 2*np.pi, 100)
    ax.fill(0.9*np.cos(theta), 0.9*np.sin(theta), color='white')
    ax.fill(0.7*np.cos(theta), 0.7*np.sin(theta), color='blue', alpha=0.3)
    ax.fill(0.5*np.cos(theta), 0.5*np.sin(theta), color='white')
    ax.fill(0.3*np.cos(theta), 0.3*np.sin(theta), color='blue', alpha=0.3)
    ax.set_title(title, fontsize=12)
    ax.set_aspect('equal')
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.axis('off')

draw_classic_srr(axes[0, 0], 'Classic Double SRR\n(Pendry 1999)')
draw_single_srr(axes[0, 1], 'Single Ring SRR')
draw_omega(axes[0, 2], 'Omega Resonator')
draw_square_srr(axes[1, 0], 'Square SRR')
draw_symmetic_srr(axes[1, 1], 'Symmetric SRR\n(Two gaps)')
draw_csrr(axes[1, 2], 'Complementary SRR\n(CSRR)')

plt.tight_layout()
plt.show()

---

## 6.7 Combining SRRs with Wire Media

To create a **Double Negative (DNG)** metamaterial:

- **Wire media** provides ε < 0 (Chapter 7)
- **SRR arrays** provide μ < 0

### The Smith et al. Experiment (2000)

The first experimental demonstration of negative index used:
- Copper SRRs on PCB substrate
- Thin copper wires between SRRs
- Operating at ~10 GHz

In [ ]:
# Visualize combined SRR + wire medium
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Effective parameters
ax1 = axes[0]

omega = np.linspace(0.01, 2.5, 500)

# SRR effective permeability
omega_0_srr = 1.0
F = 0.3
gamma_srr = 0.05
mu_eff = srr_effective_permeability(omega, omega_0_srr, F, gamma_srr)

# Wire medium effective permittivity (Drude-like)
omega_p_wire = 1.5
gamma_wire = 0.03
eps_eff = 1 - omega_p_wire**2 / (omega**2 + 1j * gamma_wire * omega)

ax1.plot(omega, eps_eff.real, 'b-', linewidth=2, label=r"$\varepsilon'$ (wire)")
ax1.plot(omega, mu_eff.real, 'r-', linewidth=2, label=r"$\mu'$ (SRR)")
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)

# Find DNG region
dng_region = (eps_eff.real < 0) & (mu_eff.real < 0)
ax1.fill_between(omega, -10, 5, where=dng_region, alpha=0.2, color='green', label='DNG region')

ax1.set_xlabel(r'$\omega / \omega_0$', fontsize=14)
ax1.set_ylabel('Effective Parameters', fontsize=14)
ax1.set_title('Combined SRR + Wire Medium', fontsize=14)
ax1.set_xlim(0, 2.5)
ax1.set_ylim(-8, 4)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Refractive index
ax2 = axes[1]

n_eff_squared = eps_eff * mu_eff
n_eff = np.sqrt(n_eff_squared)

# In DNG region, n should be negative
n_real = np.real(n_eff)
n_real[dng_region] = -np.abs(n_real[dng_region])  # Force negative in DNG

ax2.plot(omega, n_real, 'g-', linewidth=2, label="n' (real part)")
ax2.plot(omega, np.imag(n_eff), 'm--', linewidth=2, label="n'' (imaginary)")
ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax2.fill_between(omega, -5, 0, where=(n_real < 0), alpha=0.2, color='purple', label='n < 0')

ax2.set_xlabel(r'$\omega / \omega_0$', fontsize=14)
ax2.set_ylabel('Effective Refractive Index', fontsize=14)
ax2.set_title('Negative Index Region', fontsize=14)
ax2.set_xlim(0, 2.5)
ax2.set_ylim(-4, 4)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The DNG region (green) is where BOTH ε < 0 AND μ < 0.")
print("In this region, n < 0 and waves propagate with negative phase velocity!")

---

## 6.8 Summary & Key Takeaways

### What We Learned

1. **Natural materials** have μ ≈ 1 at high frequencies
2. **SRRs** are metallic LC resonators that create artificial magnetic response
3. **Resonance frequency** $\omega_0 = 1/\sqrt{LC}$ can be tuned by geometry
4. **Above resonance**, SRRs produce μ < 0
5. **Filling fraction F** controls the μ < 0 bandwidth
6. **Combining SRRs + wires** creates DNG metamaterials with n < 0

### Preview of Next Chapter

In **Chapter 7: Wire Media**, we will:
- Derive the effective permittivity of wire arrays
- Understand how to create ε < 0 at microwave frequencies
- Learn about the "plasma frequency" of wire media

---

## 📝 Exercises

1. **Design** an SRR that resonates at 5 GHz
2. **Calculate** the μ < 0 bandwidth for F = 0.5 and γ = 0.02
3. **Show** that the magnetic plasma frequency is $\omega_{mp} = \omega_0/\sqrt{1-F}$
4. **Explain** why the double-ring SRR has lower resonance than single-ring
5. **Research**: Find recent work on optical-frequency SRRs. What are the challenges?

---

**← [05 - Pendry Superlens](05_pendry_superlens.ipynb) | [07 - Wire Media](07_wire_media.ipynb) →**